# Start using the spotify API

In [1]:
import config
import spotipy
import json
import requests
from spotipy.oauth2 import SpotifyClientCredentials
import pprint
import pandas as pd
import numpy as np
import base64
import csv
import pickle

In [2]:
# Authenticate with Spotify API
client_credentials_manager = SpotifyClientCredentials(
    client_id=config.client_id,
    client_secret=config.client_secret
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
results = sp.search(q="Lose yourself",limit=1,market="GB")

In [ ]:
track_id=results["tracks"]["items"][0]["id"]
track_id

In [ ]:
# create a list containing all markets
markets = list(sp.available_markets()["markets"])

In [23]:
playlist_id_lst = pd.read_csv("list_of_playlists.csv")

In [29]:
user = "spotify"
# define function to get tracks from a playlist
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(user=username,playlist_id=playlist_id, fields="items.track.id", limit=20)
    tracks = results["items"]
    #while results['next']:
     #   results = sp.next(results)
     #   tracks.extend(results['items'])
    return tracks

In [27]:
# make list of tracks from the first x songs in each playlist contained in playlist_id_lst
tracks = []
for i, item in playlist_id_lst.iterrows():
    try:
        #print(i, item.values[0])
        tracks.extend(get_playlist_tracks(user, item.values[0]))
    except:
        pass

HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1DX5zUtQI29qCP/tracks with Params: {'limit': 10, 'offset': 0, 'fields': 'items.track.id', 'market': None, 'additional_types': 'track'} returned 404 due to Not found.


In [30]:
tracks[0]["track"]["id"]

'7oQepKHmXDaPC3rgeLRvQu'

In [34]:
# make a dataframe containing only the
df_tracks = pd.DataFrame(columns=["ID"])
for track in tracks:
    try:
        df_tracks.loc[len(df_tracks)] = track["track"]["id"]
    except:
        pass

In [31]:
df_tracks

,ID
0,7oQepKHmXDaPC3rgeLRvQu
1,7DyDjhZMEIK5Ied4juTCyc
2,7jYwZOptDPesQgzj7vhsEF
3,11eYmv0tA3wEoyD1Sad2Nv
4,54eE5H6F1HhyM5L4fRLq8s
...,...
102478,4oTeGVTZhMgn1ZwuJzroOu
102479,3B6Wm7UhJ7tmydqpdI4jod
102480,2Ew9aqhpxnNLxZgLdgdoki
102481,1RNDTmjsiekXqaOJejZaFb


In [12]:
df_tracks.to_csv("df_tracks_no_dups.csv", index_label=False)

In [14]:
# load tracks from csv into list
df_tracks = pd.read_csv("df_tracks_no_dups.csv")
#tracks = df_tracks.values.tolist()

In [28]:
df_tracks = df_tracks.drop_duplicates()
df_tracks

,ID
0,7oQepKHmXDaPC3rgeLRvQu
1,7DyDjhZMEIK5Ied4juTCyc
2,7jYwZOptDPesQgzj7vhsEF
3,11eYmv0tA3wEoyD1Sad2Nv
4,54eE5H6F1HhyM5L4fRLq8s
...,...
102478,4oTeGVTZhMgn1ZwuJzroOu
102479,3B6Wm7UhJ7tmydqpdI4jod
102480,2Ew9aqhpxnNLxZgLdgdoki
102481,1RNDTmjsiekXqaOJejZaFb


In [19]:
len(df_tracks)

70400

In [20]:
def get_features(track_id):
    track_features = []
    track_features.extend(sp.audio_features(track_id))
    return track_features

In [21]:
def write_list_to_pickle_file(my_list, filename):
    with open(filename, 'wb') as file:
        pickle.dump(my_list, file)

In [22]:
def write_list_to_csv(my_list, filename):
    with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        for item in my_list:
            writer.writerow([item])

In [ ]:
#bin_size = 10000
#ranges = [list(range(x*bin_size, (x*bin_size)+bin_size+1)) for x in range(0,10)]
for track in tracks[0:20000]:
    try:
        track_features_lst = []
        track_features_lst.extend(get_features(track))
        write_list_to_csv(track_features_lst, "track_features_lst1_4.csv")
    except:
        pass

with open("track_features_lst_2.csv","rb") as f:
    features = pickle.load(f)

In [ ]:
df = pd.read_csv("track_features_lst_2.csv")

df_features = pd.DataFrame(columns=["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"])
df_features

In [ ]:
# example DataFrame with dictionaries as rows
df = pd.read_csv("track_features_lst_2.csv")

In [ ]:
track_features = []
for playlist_id in playlist_id_lst[0:20]:
    try:
        playlist_items = sp.user_playlist_tracks(user=user, playlist_id=playlist_id, fields="items.track.id, next", limit=10)
    except:
        pass

    try:
        tracks = playlist_items['items']
        while playlist_items['next']:
            playlist_items = sp.next(playlist_items)
            tracks.extend(playlist_items['items'])
            track_features.append(sp.audio_features(tracks)[0])
    except:
        pass

# create a dataframe from the features list
df = pd.DataFrame(track_features)
#df = df[["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

In [ ]:
track_features

In [ ]:
# Get top tracks from all the playlists
for playlist_id in playlist_id_lst:
    playlist_items = sp.playlist_items(playlist_id, fields="items.track.id", limit=1)

    # now get the ID's
    track_id = []
    track_features = []
    # Print top tracks
    for i, item in enumerate(playlist_items["items"]):
        try:
            # get track_id
            track_id = item["track"]["id"]
            # get features and store them in a list
            track_features.append(sp.audio_features(track_id)[0])
        except TypeError:
            print("type error")

# create a dataframe from the features listS
df = pd.DataFrame(track_features)
df = df[["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

In [ ]:
df.head()

# code snippets

In [ ]:
# Set up authorization headers
client_id = config.client_id
client_secret = config.client_secret
auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode("ascii")).decode("ascii")
headers = {"Authorization": f"Basic {auth_header}"}

# Get access token
url = "https://accounts.spotify.com/api/token"
data = {"grant_type": "client_credentials"}
response = requests.post(url, headers=headers, data=data)
access_token = response.json()["access_token"]

# Make request to Top Tracks endpoint
artist_id = "0OdUWJ0sBjDrqHygGUXeCF"  # example artist ID
url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks"
headers = {"Authorization": f"Bearer {access_token}"}
params = {"market": "US"}

response = requests.get(url, headers=headers, params=params)

24

In [33]:
# get list of category IDs for all the markets
category_id_lst = []
markets = ["GB", "DE", "US", "BR"]
for market in markets:
    # get list of category IDs for a certain country
    categories_per_country = sp.categories(country=market, limit=10)["categories"]["items"]
    # get to the ID's of the categories and store them in a list
    for category in categories_per_country:
        if not category["id"] in category_id_lst:
            category_id_lst.append(category["id"])
        else:
            print("already in list")

already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list


In [45]:
# create a list of playlist IDs from all the categories of all the markets
playlist_id_lst = []
for category_id in category_id_lst:
    try:
        playlists_per_genre = sp.category_playlists(category_id=category_id, limit=20)["playlists"]["items"]
        try:
            for playlist in playlists_per_genre:
                if not playlist["id"] in playlist_id_lst:
                    playlist_id_lst.append(playlist["id"])
                else:
                    pass
        except:
            nothing = True
    except spotipy.SpotifyException as e:
        pass

HTTP Error for GET to https://api.spotify.com/v1/browse/categories/0JQ5DAqbMKFF1br7dZcRtK/playlists with Params: {'country': None, 'limit': 20, 'offset': 0} returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/browse/categories/0JQ5DAqbMKFKGQWi5egawY/playlists with Params: {'country': None, 'limit': 20, 'offset': 0} returned 404 due to Specified id doesn't exist
HTTP Error for GET to https://api.spotify.com/v1/browse/categories/0JQ5DAqbMKFJCTDO6ndZMP/playlists with Params: {'country': None, 'limit': 20, 'offset': 0} returned 404 due to Specified id doesn't exist
HTTP Error for GET to https://api.spotify.com/v1/browse/categories/0JQ5DAqbMKFARvdmw6EIef/playlists with Params: {'country': None, 'limit': 20, 'offset': 0} returned 404 due to Specified id doesn't exist


In [48]:
playlist_id_lst

['37i9dQZF1DX4jP4eebSWR9',
 '37i9dQZF1DX7ZUug1ANKRP',
 '37i9dQZF1DX36edUJpD76c',
 '37i9dQZF1DWUW2bvSkjcJ6',
 '37i9dQZF1DX4dyzvuaRJ0n',
 '37i9dQZF1DX0XUsuxWHRQd',
 '37i9dQZF1DXcF6B6QPhFDv',
 '37i9dQZF1DX4SBhb3fqCJd',
 '37i9dQZF1DWWjGdmeTyeJ6',
 '37i9dQZEVXbJiZcmkrIHGU',
 '37i9dQZEVXbMDoHDwVN2tF',
 '37i9dQZEVXbLiRSasKsNU9',
 '37i9dQZEVXbNv6cjoMVCyg',
 '37i9dQZF1DWX7rdRjOECPW',
 '37i9dQZF1DX2ogDiL6nZJr',
 '37i9dQZF1DWURW6ov0F5Ak',
 '37i9dQZF1DX05r4Oy3Ln97',
 '37i9dQZF1DXbwoaqxaoAVr',
 '37i9dQZF1DXd1MXcE8WTXq',
 '37i9dQZF1DX4Y4RhrZqHhr',
 '37i9dQZF1DWZSdcRHMl2tT',
 '37i9dQZF1DXbtuVQL4zoey',
 '37i9dQZF1DWZ7eJRBxKzdO',
 '37i9dQZF1DX83I5je4W4rP',
 '37i9dQZF1DWUajed02NzWR',
 '37i9dQZF1DX5FuBDzVtEFX',
 '37i9dQZF1DWUYPasrIWf5j',
 '37i9dQZF1DXa8n42306eJB',
 '37i9dQZF1DXaSYtampkg5n',
 '37i9dQZF1DX8NMUtC3b3gL',
 '37i9dQZF1DX9FIMhEujaK6',
 '37i9dQZF1DX6QlnUtJCO12',
 '37i9dQZF1DWTKxc7ZObqeH',
 '37i9dQZF1DWZESE3fHLhmD',
 '37i9dQZF1DX3Ebqev5IkYU',
 '37i9dQZF1DXaIrEwuQ3hyy',
 '37i9dQZF1DWSWyJydK4fTU',
 

In [ ]:
df_playlists = pd.DataFrame(playlist_id_lst)
df_playlists.to_csv("list_of_playlists.csv", index_label=False)

In [ ]:
# Get "Today's Top Hits" playlist ID
playlist_id = "37i9dQZF1DXcBWIGoYBM5M"

# Get top tracks and show only the ids
playlist_items = sp.playlist_items(playlist_id, fields="items.track.id", limit=10)

track_id = []
track_features = []
# Print top tracks
for i, item in enumerate(playlist_items["items"]):
    # get track_id
    track_id = item["track"]["id"]
    # get features and store them in a list
    track_features.append(sp.audio_features(track_id)[0])

# create a dataframe from the features list
df = pd.DataFrame(track_features)
df = df[["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"]]